In [4]:
import sys
sys.path.append('../../')
from Model import Convlstm as clm
import torch
import torch.nn as nn
from einops import rearrange
import torchinfo
from Utils.Functions import get_Net
print(5)


In [16]:
class ConvLSTMCompose(nn.Module):
    @staticmethod
    def add_model_specific_args(parent_parser):
        parser = parent_parser.add_argument_group("LitModel")
        parser.add_argument("--input_dim", type=int, default=1430)

        return parent_parser
    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers, batch_first, bias, return_all_layers):
        super(ConvLSTMCompose, self).__init__()
        self.lstm = clm.ConvLSTM(input_dim=input_dim, hidden_dim=hidden_dim, kernel_size=kernel_size,
                                 num_layers=num_layers, batch_first=batch_first, bias=bias,
                                 return_all_layers=return_all_layers)
        self.batch_norm = nn.BatchNorm2d(hidden_dim)
        if hidden_dim>=100 and hidden_dim<384:
            self.avg_pool= nn.AdaptiveAvgPool2d((2, 3))
        elif hidden_dim>=384:
            self.avg_pool= nn.AdaptiveAvgPool2d((2, 1))
        elif hidden_dim==48:
            self.avg_pool= nn.AdaptiveAvgPool2d((4, 4))
        else:
            self.avg_pool= nn.AdaptiveAvgPool2d((2, 4))
        self.flatten = nn.Flatten()


    def forward(self, x):
        print(x.shape)
        x = rearrange(x, 'b c (w1 w2) h -> b w1 c w2 h', w1=5, w2=13)
        print(x.shape)
        x = self.lstm(x)[0][0][:, -1, :, :, :]
        print(x.shape)
        x = self.flatten(self.avg_pool(self.batch_norm(x)))
        return x

In [19]:
torchinfo.summary(ConvLSTMCompose(22, 48, (3, 3), 1, True, True, False), (96, 22, 65, 21))

In [3]:
torchinfo.summary(clm.ConvLSTMCompose(22, 96, (3, 3), 2, True, True, False), (96, 22, 65, 21))